In [46]:
import os
import csv
import re
from pymongo import MongoClient
from pprint import pprint

# Credentials

In [2]:
pw_file = 'credentials/mongo_pw.txt'
if os.path.exists(pw_file): 
    with open(pw_file, 'r') as f:
        pub_ip, mongo_usr, mongo_usr_pw = f.readline().strip().split(', ')

# Connect to MongoDB

In [3]:
# connect to ec2 mongo client
client = MongoClient('{0}:27017'.format(pub_ip))

# get reference to  resume_db
db = client['github_db']

# authenticate user for database
db.authenticate(mongo_usr, mongo_usr_pw)

True

# Show MongoDB collections

In [4]:
def show_collections():
    '''
    IN: None
    RETURN: Printed output of collections and collection counts
    '''
    for name in sorted(db.collection_names()):
        print('Documents in \"{0}\" collection: {1}'.format(name, db[name].count()))

In [5]:
show_collections()

Documents in "git_repos_contributors" collection: 1985
Documents in "git_repos_docs" collection: 84495
Documents in "git_repos_meta" collection: 1985
Documents in "git_repos_subscribers" collection: 1985
Documents in "git_users_followers" collection: 456
Documents in "git_users_following" collection: 456
Documents in "git_users_meta" collection: 456


In [123]:
def get_fieldname_tsv(col):
    cursor = db[col].find_one()
    fields = list(cursor.keys())
    fields = '\t '.join(fields) + '\n'
    
    if col == 'git_repos_docs':
        fields = fields.replace('file_contents', 'strip_contents')

    return fields

In [124]:
get_fieldname_tsv('git_repos_docs')

'_id\t repo_name\t repo_id\t file_name\t repo_fullname\t doc_name\t strip_contents\t repo_owner_login\t file_extension\n'

In [136]:
def convert_repodoc_tsv(doc):
    doc['file_contents'] = doc['file_contents'].replace('\n', ' ')
    doc['file_contents'] = doc['file_contents'].replace('\t', ' ')
    doc['file_contents'] = re.sub('\s+', ' ', doc['file_contents']).strip()
    
    doc_str = [str(v) for k,v in doc.items()]
    doc_str = '\t '.join(doc_str) + '\n'
    return doc_str

In [137]:
cursor = db['git_repos_docs'].find_one()
convert_repodoc_tsv(cursor)

"57ce15d48532183ad7e11e84\t awesome-python\t 21289110\t PULL_REQUEST_TEMPLATE.md\t vinta/awesome-python\t 21289110__PULL_REQUEST_TEMPLATE.md\t ## What is this Python project? Describe features. ## What's the difference between this Python project and similar ones? Enumerate comparisons. -- Anyone who agrees with this pull request could vote for it by adding a :+1: to it, and usually, the maintainer will merge it when votes reach **20**.\t vinta\t md\n"

In [155]:
def filter_filetypes(db, col, _filter='all'):
    
    if _filter != 'all':
        db_new = db[col].find({'file_extension': re.compile(_filter, re.IGNORECASE)})
    else:
        db_new = db[col].find()
    
    return db_new

In [159]:
filtered_col = filter_filetypes(db, 'git_repos_docs')

In [195]:
def export_mongo_to_tsv(db, col, filename, _filter='all'):
    
    cursor = filter_filetypes(db, col, _filter)
    ct = 0
    
    with open('{0}.txt'.format(filename), 'w') as outfile:
        #if ct < 1:
        #    outfile.write(get_fieldname_tsv(col))

        for doc in cursor:
            csv_str = convert_repodoc_tsv(doc)
            outfile.write(csv_str)

In [196]:
export_mongo_to_tsv(db, 'git_repos_docs', 'repo_docs', 'py')

In [190]:
import json

with open('{0}.json'.format('xxxx'), 'w') as outfile:

        for doc in filter_filetypes(db, 'git_repos_docs', 'py')[:200]:
            json.dump(doc, outfile, default=)

# Upload to S3

In [204]:
with open('credentials/aws.csv', 'r') as infile:
    _ = infile.readline()
    username, access_key, secret_key = infile.readline().replace('"', '').split(',')

In [214]:
import tinys3

conn = tinys3.Connection(access_key, secret_key, tls=True)

f = open('repo_py_docs.txt','rb')
conn.upload('repo_py_docs.txt',f,'github-spark')

<Response [200]>